In [1]:
from database.strategy import Strategy
from database.sec import SEC
from database.market import Market
from transformer.date_transformer import DateTransformer
from transformer.column_transformer import ColumnTransformer
from transformer.model_transformer import ModelTransformer
from transformer.product_transformer import ProductTransformer
from transformer.predictor_transformer import PredictorTransformer
from preprocessor.model_preprocessor import ModelPreprocessor
from preprocessor.predictor_preprocessor import PredictorPreprocessor
from modeler.modeler import Modeler as sp
from utils.date_utils import DateUtils
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime, timedelta, timezone
from tqdm import tqdm
import math

In [52]:
## Loading Constants
start = "2008-01-01"
end = datetime(2021,1,1).strftime("%Y-%m-%d")
# Loading Databases
strat_db = Strategy("unity")
market = Market()
sec = SEC("sec")
market.connect()
tickers = market.retrieve_data("sp500").sort_values("Symbol")
market.close()
quarterly_range = range(1,5)
yearly_range = range(2018,2021)
model_range = range(len(tickers["Symbol"]))
reload = False

In [3]:
if reload:
    market.connect()
    market.drop_table("weekly_ranked")
    deltas = []
    for  i in tqdm(range(len(tickers["Symbol"]))):
        ticker = tickers.iloc[i]["Symbol"]
        price = market.retrieve_price_data("prices",ticker)
        price = DateTransformer.convert_to_date(ColumnTransformer.rename_columns(price," "),"date")
        price["year"] = [x.year for x in price["date"]]
        price["week"] = [x.week for x in price["date"]]
        price["quarter"] = [x.quarter for x in price["date"]]
        grouped = price.groupby(["year","quarter","week"]).mean().reset_index()
        grouped["delta"] = grouped["adjclose"].pct_change()
        grouped["ticker"] = ticker
        grouped.dropna(inplace=True)
        deltas.append(grouped)
    d = pd.concat(deltas)
    ranks = []
    for year in tqdm(d["year"].unique()):
        yearly_deltas = d[d["year"] == year]
        for week in yearly_deltas["week"].unique():
            weekly_deltas = yearly_deltas[yearly_deltas["week"] == week]
            sorted_deltas = weekly_deltas.sort_values("delta",ascending=True).reset_index(drop=True)
            sorted_deltas["rank"] = sorted_deltas.index + 1
            ranks.append(sorted_deltas)
    weekly_ranked = pd.concat(ranks)
    market.store_data("weekly_ranked",weekly_ranked)    
    classification = market.retrieve_data("dataset_week_classification")
    regression = market.retrieve_data("dataset_week_regression")
    market.close()
else:
    market.connect()
    weekly_ranked = market.retrieve_data("weekly_ranked")
    classification = market.retrieve_data("dataset_week_classification")
    regression = market.retrieve_data("dataset_week_regression")
    market.close()

In [39]:
regression["quarter"]  = [math.floor(row[1]["week"]/ 14) + 1 for row in regression.iterrows()]

In [40]:
regression["quarter"].unique()

array([1, 2, 3, 4], dtype=int64)

In [41]:
strat_db.connect()
bond_sim = strat_db.retrieve_data("bond_quarterly_sim")
fund_sim = strat_db.retrieve_data("fundamental_quarterly_sim")
weekly_sim = strat_db.retrieve_data("weekly_sim")
strat_db.close()

In [42]:
weekly_sim

,_id,year,quarter,week,weekly_price_classification_prediction,weekly_price_classification_score,weekly_price_regression_prediction,weekly_price_regression_score,ticker
0,6007b8047d53c0b09cc7c609,2018,1,2,0.0,0.528302,64.710640,0.021251,A
1,6007b8047d53c0b09cc7c60a,2018,1,3,1.0,0.528302,63.552856,0.021251,A
2,6007b8047d53c0b09cc7c60b,2018,1,4,1.0,0.528302,64.219604,0.021251,A
3,6007b8047d53c0b09cc7c60c,2018,1,5,1.0,0.528302,65.173843,0.021251,A
4,6007b8047d53c0b09cc7c60d,2018,1,6,0.0,0.528302,65.793182,0.021251,A
...,...,...,...,...,...,...,...,...,...
60275,60080b157d53c0b09cc8b17c,2020,3,37,0.0,0.566038,34.249821,0.025693,ZION
60276,60080b157d53c0b09cc8b17d,2020,3,38,1.0,0.566038,34.259735,0.025693,ZION
60277,60080b157d53c0b09cc8b17e,2020,3,39,0.0,0.566038,36.289635,0.025693,ZION
60278,60080b157d53c0b09cc8b17f,2020,3,40,1.0,0.566038,34.798912,0.025693,ZION


In [43]:
final = weekly_sim.drop("_id",axis=1).merge(
    fund_sim.drop("_id",axis=1).merge(bond_sim.drop("_id",axis=1),on=["year","quarter","ticker"],how="left")
    ,on=["year","quarter","ticker"],how="left")

In [53]:
sims = []
gap = 90
quarter_gap = int(gap/90)
training_years = 3
sims = []
sec.connect()
strat_db.connect()
market.connect()
strat_db.drop_table("ranked_weekly_sim")
strat_db.drop_table("ranked_weekly_sim_errors")
for year in tqdm(yearly_range):
    for quarter in tqdm(quarterly_range):
        try:
            ## Setting Up
            print(year,quarter)
            date_ranges = DateUtils.create_quarterly_training_range_rec(year,quarter,training_years,gap)
            training_start,training_end,prediction_start,prediction_end = date_ranges
            dates = pd.to_datetime(date_ranges)
            quarters = [x.quarter for x in dates]
            weeks = [x.week for x in dates]
            years = [x.year for x in dates]
            print(date_ranges)
            ### switch to all tickers
            for i in model_range:
                try:
                    ticker = tickers.iloc[i]["Symbol"].replace(".","-")
                    price = market.retrieve_price_data("prices",ticker)
                    rank = weekly_ranked[weekly_ranked["ticker"] == ticker]
                    fqd = final[final["ticker"]==ticker]
                    mt = ModelTransformer(ticker,training_start,training_end,gap)
                    mr = ModelPreprocessor(ticker)
                    prot = ProductTransformer(ticker,prediction_start,prediction_end)
                    ## ranking_model
#                     rank_data = rank.drop(["_id","ticker","adjclose"],axis=1,errors="ignore")\
#                                     .merge(regression.copy().drop(["_id","quarter"],axis=1),on=["year","week"],how="left") \
#                                     .merge(fqd.drop(["quarter","ticker"],axis=1),on=["year","week"],how="right")
#                     rkpd = mt.weekly_rank_transform(rank_data,ticker,years[0],weeks[0]
#                                                        ,years[1],weeks[1],quarter_gap)
#                     kpd = mr.rank_preprocess(rkpd.copy(),ticker)
#                     rkfr = sp.regression(kpd,ranked=True,tf=True)
                    ## regression_model
#                     first_index =  regression_data[(regression_data["year"] == years[0]) & 
#                                            (regression_data["week"] == weeks[0])].index.values.tolist()[0]
#                     last_index = regression_data[(regression_data["year"] == years[1]) & 
#                                            (regression_data["week"] == weeks[1])].index.values.tolist()[-1]
#                     rqpd = regression_data.iloc[first_index:last_index+1]
#                     rqpd["y"] = rqpd[ticker]
#                     rpd = mr.day_trade_preprocess_regression(rqpd.copy(),ticker)
#                     rpr = sp.regression(rpd,ranked=False,tf=True)
                    ## classification_model
                    classification_data = rank.drop(["_id","ticker"],axis=1,errors="ignore") \
                                                .merge(classification.drop("_id",axis=1),on=["year","week"],how="left") \
                                                .merge(fqd.drop(["quarter","ticker"],axis=1),on=["year","week"],how="left")
                    first_index =  classification_data[(classification_data["year"] == years[0]) & 
                                           (classification_data["week"] == weeks[0])].index.values.tolist()[0]
                    last_index = classification_data[(classification_data["year"] == years[1]) & 
                                           (classification_data["week"] == weeks[1])].index.values.tolist()[-1]
                    cqpd = classification_data.iloc[first_index:last_index+1]
                    cqpd["y"] = cqpd[ticker]
                    cpd = mr.day_trade_preprocess_classify(cqpd.copy(),ticker)
                    cpr = sp.classification(cpd,tf=False)
                    fundamental_results = pd.DataFrame([cpr])
                    product_refineds = []
                    product_qpds = []
                    for i in range(len(fundamental_results)):
                        fundamental_result = fundamental_results.iloc[i]
                        if fundamental_result["model_type"] == "regression":
                            product_data = mt.quarterly_price_transform(regression_data,ticker
                                                                    ,years[2],quarters[2],
                                                                   years[3],quarters[3],quarter_gap)
                            product_refined = mr.day_trade_preprocess_regression(product_data.copy(),ticker)
                        elif fundamental_result["model_type"] == "classification":
                            product_data = mt.quarterly_price_transform(classification_data,ticker
                                                                    ,years[2],quarters[2],
                                                                   years[3],quarters[3],quarter_gap)
                            product_refined = mr.day_trade_preprocess_classify(product_data.copy(),ticker)
                        elif fundamental_result["model_type"] == "ranked":
                            product_data = mt.quarterly_rank_transform(rank_data,ticker,years[2],quarters[2]
                                                           ,years[3],quarters[3],quarter_gap)
                            product_refined = mr.rank_preprocess(product_data,ticker)
                        product_refineds.append(product_refined)
                    product_weeks = product_data["week"]
                    sim = prot.merge_weeklies_v2(product_refineds,fundamental_results,product_weeks,"ranked")
                    strat_db.store_data("ranked_weekly_sim",pd.DataFrame(sim))
                except Exception as e:
                    message = {"status":"quarterly modeling","ticker":ticker,"year":str(year),"quarter":str(quarter),"message":str(e)}
                    print(message)
                    strat_db.store_data("ranked_quarterly_sim_errors",pd.DataFrame([message]))
        except Exception as e:
            print(year,quarter,str(e))
sec.close()
market.close()
strat_db.close()

  0%|                                                                                                                                                                                                     | 0/4 [00:00<?, ?it/s]

2018 1
['2015-01-01', '2018-01-01', '2018-01-01', '2018-04-02']
{'status': 'quarterly modeling', 'ticker': 'ABBV', 'year': '2018', 'quarter': '1', 'message': "'ABBV'"}
{'status': 'quarterly modeling', 'ticker': 'ALLE', 'year': '2018', 'quarter': '1', 'message': "'ALLE'"}
{'status': 'quarterly modeling', 'ticker': 'AMCR', 'year': '2018', 'quarter': '1', 'message': "'AMCR'"}
{'status': 'quarterly modeling', 'ticker': 'ANET', 'year': '2018', 'quarter': '1', 'message': "'ANET'"}
{'status': 'quarterly modeling', 'ticker': 'APTV', 'year': '2018', 'quarter': '1', 'message': "'APTV'"}
{'status': 'quarterly modeling', 'ticker': 'AVGO', 'year': '2018', 'quarter': '1', 'message': "'AVGO'"}
{'status': 'quarterly modeling', 'ticker': 'AWK', 'year': '2018', 'quarter': '1', 'message': "'AWK'"}
{'status': 'quarterly modeling', 'ticker': 'CARR', 'year': '2018', 'quarter': '1', 'message': 'list index out of range'}
{'status': 'quarterly modeling', 'ticker': 'CBOE', 'year': '2018', 'quarter': '1', 'messa


 25%|███████████████████████████████████████████████                                                                                                                                             | 1/4 [13:59<41:58, 839.53s/it]

{'status': 'quarterly modeling', 'ticker': 'ZTS', 'year': '2018', 'quarter': '1', 'message': "'ZTS'"}
2018 2
['2015-04-01', '2018-04-02', '2018-04-02', '2018-07-02']
{'status': 'quarterly modeling', 'ticker': 'ABBV', 'year': '2018', 'quarter': '2', 'message': "'ABBV'"}
{'status': 'quarterly modeling', 'ticker': 'ALLE', 'year': '2018', 'quarter': '2', 'message': "'ALLE'"}
{'status': 'quarterly modeling', 'ticker': 'AMCR', 'year': '2018', 'quarter': '2', 'message': "'AMCR'"}
{'status': 'quarterly modeling', 'ticker': 'ANET', 'year': '2018', 'quarter': '2', 'message': "'ANET'"}
{'status': 'quarterly modeling', 'ticker': 'APTV', 'year': '2018', 'quarter': '2', 'message': "'APTV'"}
{'status': 'quarterly modeling', 'ticker': 'AVGO', 'year': '2018', 'quarter': '2', 'message': "'AVGO'"}
{'status': 'quarterly modeling', 'ticker': 'AWK', 'year': '2018', 'quarter': '2', 'message': "'AWK'"}
{'status': 'quarterly modeling', 'ticker': 'CARR', 'year': '2018', 'quarter': '2', 'message': 'list index ou


 50%|██████████████████████████████████████████████████████████████████████████████████████████████                                                                                              | 2/4 [29:18<28:46, 863.46s/it]

...

{'status': 'quarterly modeling', 'ticker': 'ZTS', 'year': '2020', 'quarter': '2', 'message': "'ZTS'"}
2020 3
['2017-07-03', '2020-06-30', '2020-07-01', '2020-09-30']
{'status': 'quarterly modeling', 'ticker': 'ABBV', 'year': '2020', 'quarter': '3', 'message': "'ABBV'"}
{'status': 'quarterly modeling', 'ticker': 'ALLE', 'year': '2020', 'quarter': '3', 'message': "'ALLE'"}
{'status': 'quarterly modeling', 'ticker': 'AMCR', 'year': '2020', 'quarter': '3', 'message': "'AMCR'"}
{'status': 'quarterly modeling', 'ticker': 'ANET', 'year': '2020', 'quarter': '3', 'message': "'ANET'"}
{'status': 'quarterly modeling', 'ticker': 'APTV', 'year': '2020', 'quarter': '3', 'message': "'APTV'"}
{'status': 'quarterly modeling', 'ticker': 'AVGO', 'year': '2020', 'quarter': '3', 'message': "'AVGO'"}
{'status': 'quarterly modeling', 'ticker': 'AWK', 'year': '2020', 'quarter': '3', 'message': "'AWK'"}
{'status': 'quarterly modeling', 'ticker': 'CARR', 'year': '2020', 'quarter': '3', 'message': 'list index ou


 75%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████                                               | 3/4 [41:27<13:40, 820.73s/it]

{'status': 'quarterly modeling', 'ticker': 'ZTS', 'year': '2020', 'quarter': '3', 'message': "'ZTS'"}
2020 4
['2017-10-02', '2020-09-30', '2020-10-01', '2020-12-31']
{'status': 'quarterly modeling', 'ticker': 'ABBV', 'year': '2020', 'quarter': '4', 'message': "'ABBV'"}
{'status': 'quarterly modeling', 'ticker': 'ALLE', 'year': '2020', 'quarter': '4', 'message': "'ALLE'"}
{'status': 'quarterly modeling', 'ticker': 'AMCR', 'year': '2020', 'quarter': '4', 'message': "'AMCR'"}
{'status': 'quarterly modeling', 'ticker': 'ANET', 'year': '2020', 'quarter': '4', 'message': "'ANET'"}
{'status': 'quarterly modeling', 'ticker': 'APTV', 'year': '2020', 'quarter': '4', 'message': "'APTV'"}
{'status': 'quarterly modeling', 'ticker': 'AVGO', 'year': '2020', 'quarter': '4', 'message': "'AVGO'"}
{'status': 'quarterly modeling', 'ticker': 'AWK', 'year': '2020', 'quarter': '4', 'message': "'AWK'"}
{'status': 'quarterly modeling', 'ticker': 'CARR', 'year': '2020', 'quarter': '4', 'message': 'list index ou


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3/3 [2:42:24<00:00, 3248.09s/it]

{'status': 'quarterly modeling', 'ticker': 'ZTS', 'year': '2020', 'quarter': '4', 'message': "'ZTS'"}
